In [ ]:
#В этом ноутбуке находится код создания векторного хранилица для одного года.

In [3]:
import pandas as pd
f = pd.read_csv('/home/jupyter/datasphere/project/rag_east_data.csv')
filtered_df = f[f['year'] == 2016]  # или любой другой год из 2017, 2019, 2022, 2023

docs = filtered_df['translated_text']
docs

20843    «Синьхуа -агентство» сообщило, что, согласно о...
20844    Проект поправки к трудовым законодательству ещ...
20845    ／ Reporter Fang Weiduo / Taipei сообщает, что ...
20846    В первый день начала школы американских школ С...
20847    Credit Suisse заявила, что он согласился запла...
                               ...                        
37043    Стресс на рабочем месте, как говорят, является...
37044    Россия чувствует, что демократическая команда ...
37045    В 2006 году участники проекта НЛО Хантер изуча...
37046    Студенты показали, что многие женщины сталкива...
37047    Турецкие ВВС были успешно разрушены Дайсом (Ис...
Name: translated_text, Length: 981, dtype: object

In [ ]:
%pip install yandex_chain 
%pip install -U langchain-community
%pip install faiss-cpu

In [5]:
import os
import faiss
import time
import pickle
from tqdm import tqdm
from yandex_chain import YandexLLM, YandexEmbeddings
from langchain.vectorstores import FAISS
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def split_document(doc, max_tokens=1200):
    words = word_tokenize(doc)
    return [' '.join(words[i:i + max_tokens]) for i in range(0, len(words), max_tokens)]

# Настройка Yandex Embeddings
folder_id = os.environ['folder_id']
api_key = os.environ['my-API']
embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key)


docs_with_progress = []
for i, doc in enumerate(tqdm(docs, desc="Обработка документов", unit="документ")):
    try:
        if len(word_tokenize(doc)) > 1200:     # у эмбеддингов ограничение 2048 токенов (сабтокены), а у nltk по-другому
            split_docs = split_document(doc)
            docs_with_progress.extend(split_docs)
        else:
            docs_with_progress.append(doc)
    except Exception as e:
        print(f"Ошибка при обработке документа {i}: {e}")
        continue
print(len(docs_with_progress))

start_time = time.time()
vectorstore = None
batch_size = 100  # Количество документов а батче
current_batch = []
batch_index = 0

for i, doc in enumerate(tqdm(docs_with_progress, desc="Создание векторного хранилища", unit="документ")):
    current_batch.append(doc)
    if len(current_batch) >= batch_size or i == len(docs_with_progress) - 1:
        try:
            if vectorstore is None:
                vectorstore = FAISS.from_texts(current_batch, embedding=embeddings)
            else:
                vectorstore.add_texts(current_batch)
        except Exception as e:
            print(f"Ошибка на итерации {i} при добавлении документа в векторное хранилище: {e}")
            continue
            
            # Сохранение промежуточных результатов
        faiss.write_index(vectorstore.index, f"faiss_2016_index.index")
        with open(f"faiss_2016_index.pkl", "wb") as f:
            pickle.dump(vectorstore, f)

        current_batch = []  # Очистка текущей партии
        batch_index += 1  # Увеличение индекса батча
            


end_time = time.time()
vectorstore_time = end_time - start_time
print(f"Время на выполнение vectorstore_time: {vectorstore_time/60} минут")

print("Готово")



[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Обработка документов: 100%|██████████| 981/981 [00:00<00:00, 1532.41документ/s]


984


Создание векторного хранилища: 100%|██████████| 984/984 [18:06<00:00,  1.10s/документ]

Время на выполнение vectorstore_time: 18.107376035054525 минут
Готово
